In [17]:
import numpy as np
import matplotlib.pyplot as plt

In [18]:
N = 6 # number of players
nr_agents_per_player = 100 # number of agents per player
fitnesses = np.sort(np.random.rand(N))[::-1] # fitnesses, sorted
generations = 5000  # number of generations
mutation_rate = 0.5  # Probability of mutation
mutation_amount = 0.3  # Amount of mutation
populations = np.array([np.random.rand(nr_agents_per_player ) for _ in range(N)]) # N times nr_agents_per_player table

In [19]:
def compute_payoffs(populations):
    scores = np.zeros((N, nr_agents_per_player))  # Scores for each member
    #print(scores)
    for i in range(N):
        for j in range(nr_agents_per_player):
            for l in range(1):
                action = 1*(np.random.rand() < populations[i][j])
                #action = (1 if populations[i][j] > 0.5 else 0)  # Action (1 or 0) of the i-th player's j-th member, based on probability
                #print(action)
                #other_actions = [np.random.rand() < populations[o][np.random.randint(nr_agents_per_player)]
                #             for o in range(N) if o != i]
                other_actions = [1*(np.mean(populations[o])>0.5)
                                for o in range(N) if o != i]
                #print(other_actions)
                #print(other_actions)
                k_value = max(1, sum(other_actions) + action)
                #print(k_value)
                if k_value >= i+1:
                    scores[i, j] += N/k_value
                else:
                    scores[i, j] += 0
    return scores

In [20]:
def reproduce(populations, scores):
    new_populations = []
    for i in range(N):
        # Select members with scores proportional to their performance
        #print(scores[i])
        if sum(scores[i]) > 0:
            probabilities = scores[i] / scores[i].sum()
        else:
            probabilities = np.ones(nr_agents_per_player) / nr_agents_per_player
        #print(probabilities)
        selected_indices = np.random.choice(np.arange(nr_agents_per_player), size=nr_agents_per_player,
                                             p=probabilities)
        new_population = populations[i][selected_indices]
        #print(new_population)
        # Apply mutation
        mutations = np.random.rand(nr_agents_per_player) < mutation_rate
        mutation_values = (np.random.rand(nr_agents_per_player) - 0.5) * 2 * mutation_amount
        new_population[mutations] += mutation_values[mutations]
        new_population = np.clip(new_population, 0, 1)  # Ensure probabilities are in [0, 1]
        #print(new_population)
        new_populations.append(new_population)
    return np.array(new_populations)

In [21]:
from collections import Counter
N = 4
combination_counts4= Counter()
transition_counts4 = Counter()
for i in range(1):
    populations = np.array([np.random.rand(nr_agents_per_player ) for _ in range(N)]) 
    fitnesses = np.sort(np.random.rand(N))[::-1] 
    prev_actions = None
    # Simulation
    for gen in range(generations):
        scores = compute_payoffs(populations)
        populations = reproduce(populations, scores)
        other_actions = tuple(1 * (np.mean(populations[o]) > 0.5) for o in range(N))
        #print(other_actions)
        if gen > 500:
            combination_counts4[other_actions] += 1
            if prev_actions is not None:
                transition_counts4[(prev_actions, other_actions)] += 1
        prev_actions = other_actions

# Results
final_pops = [pop.mean() for pop in populations]
#for i, mean_p in enumerate(final_pops):
    #print(f"Player {i + 1} mean p: {mean_p:.3f}")

print("Combination counts:")
print(combination_counts4)

print("Transition counts:")
print(transition_counts4)

Combination counts:
Counter({(np.int64(0), np.int64(1)): 2641, (np.int64(0), np.int64(0)): 1817, (np.int64(1), np.int64(1)): 23, (np.int64(1), np.int64(0)): 18})
Transition counts:
Counter({((np.int64(0), np.int64(1)), (np.int64(0), np.int64(1))): 2402, ((np.int64(0), np.int64(0)), (np.int64(0), np.int64(0))): 1578, ((np.int64(0), np.int64(1)), (np.int64(0), np.int64(0))): 239, ((np.int64(0), np.int64(0)), (np.int64(0), np.int64(1))): 220, ((np.int64(0), np.int64(0)), (np.int64(1), np.int64(0))): 18, ((np.int64(1), np.int64(1)), (np.int64(0), np.int64(1))): 14, ((np.int64(1), np.int64(0)), (np.int64(1), np.int64(1))): 13, ((np.int64(1), np.int64(1)), (np.int64(1), np.int64(1))): 9, ((np.int64(1), np.int64(0)), (np.int64(0), np.int64(1))): 5, ((np.int64(0), np.int64(0)), (np.int64(1), np.int64(1))): 1})


In [31]:
weighted_sum4 = np.zeros(len(next(iter(combination_counts4))))  # Initialize with the lengths of the keys
for key, weight in combination_counts4.items():
    weighted_sum4 += np.array(key) * weight

print("Weighted sum:", weighted_sum4/4500)

Weighted sum: [0.         0.69666667 0.92933333 0.52622222]
